In [2]:
import andes
import os

from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import gurobipy as gb
import pandas as pd
import numpy as np
import logging
logger = logging.getLogger(__name__)

## Test OPF model created by Jinning Wang


### Based system
Import data from andes instant

In [3]:
class system:
    """
    Base class of jams system.
    
    Parameters
    ----------
    name : str
        Name of the system.

    Attributes
    ----------
    bus: pandas.DataFrame
        Bus data.
    gen: pandas.DataFrame
        Generator data.
    line: pandas.DataFrame
        Line data.
    load: pandas.DataFrame
        Load data.
    gen_gsf: pandas.DataFrame
        Generator shift factor of gen bus data.
    cost: pandas.DataFrame
        Cost data.
    """

    def __init__(self, name='system'):
        self.name = name

    def update_dict(self, model=None):
        """
        Update model DataFrame into model dict.

        Parameters
        ----------
        model : list
            list of models that need to be updated.
            If None is given, update all models.
        """
        # --- validity check ---
        if not hasattr(self, 'cost'):
            self._default_cost()

        # --- build dict ---
        if not model:
            mdl_list = ['bus', 'gen', 'line', 'gen_gsf', 'cost']
        else:
            mdl_list = model
        for mdl in mdl_list:
            mdl_df = getattr(self, mdl).copy()
            mdl_df.set_index(['idx'], inplace=True)
            setattr(self, mdl+'dict', mdl_df.T.to_dict())

    def from_andes(self, ssa):
        """
        Create jams system from ANDES system.

        Parameters
        ----------
        ssa : andes.system.system
            ANDES system.
            
        Notes
        -----
        All generators are set as controllable.
        """
        # --- base mva ---
        self.mva = ssa.config.mva

        # --- bus ---
        bus_cols = ['idx', 'u', 'name', 'Vn', 'vmax', 'vmin', 'v0', 'a0', 'area', 'zone', 'owner']
        self.bus = ssa.Bus.as_df()[bus_cols]
        self.bus.sort_values('idx', inplace=True)

        # --- generator ---
        stg_cols = ['idx', 'u', 'name', 'Sn', 'Vn', 'bus', 'p0',
                    'pmax', 'pmin', 'v0']
        self.gen = build_group_table(ssa, 'StaticGen', stg_cols).reset_index(drop=True)
        self.gen['ctrl'] = 1
        # TODO: later on, merge 'ramp5', 'ramp10', 'ramp30'
        self.gen['ramp5'] = 100
        self.gen['ramp10'] = 200
        self.gen['ramp30'] = 600
        # --- later on ---
        # self.gen['ramp5'] = self.gen['ramp5'] / self.mva
        # self.gen['ramp10'] = self.gen['ramp10'] / self.mva
        # self.gen['ramp30'] = self.gen['ramp30'] / self.mva
        # if self.gen['ramp5'].max() == 0:
        #     self.gen['ramp5'] = 100
        #     self.gen['ramp10'] = 100
        #     self.gen['ramp30'] = 100

        # --- load ---
        pq_cols = ['idx', 'u', 'name', 'bus', 'Vn', 'p0', 'q0',
                   'vmax', 'vmin', 'owner']
        self.load = ssa.PQ.as_df()[pq_cols]
        self.load.sort_values(by='idx', inplace=True)

        # --- line ---
        line_cols = ['idx', 'u', 'name', 'bus1', 'bus2', 'Sn', 'fn', 'Vn1', 'Vn2',
                     'trans', 'tap', 'phi', 'rate_a', 'rate_b', 'rate_c']
        ssa_line = ssa.Line.as_df()
        self.line = ssa_line[line_cols][ssa_line['trans'] == 0].reset_index(drop=True)

        self.load.sort_values(by='idx', inplace=True)
        if self.line['rate_a'].max() == 0:
            self.line['rate_a'] = 2000
            self.line['rate_b'] = 2000
            self.line['rate_c'] = 2000
        self.line['rate_a'] = self.line['rate_a'] / self.mva
        self.line['rate_b'] = self.line['rate_b'] / self.mva
        self.line['rate_c'] = self.line['rate_c'] / self.mva

        # --- GSF ---
        ssp = to_pandapower(ssa)
        gsf_matrix = make_GSF(ssp)
        self.gsf_matrix = gsf_matrix
        gsfdata = pd.DataFrame(gsf_matrix)
        
        gsfdata.columns = self.bus['idx']
        gsfdata['line'] = self.line.idx
        gsfdata.set_index('line', inplace=True)
        gsfT = gsfdata.T
        gsfT['bus'] = self.bus['idx']
        self.gen_gsf = self.gen[['idx', 'name', 'bus']].merge(gsfT, on='bus', how='left')
        self.gen_gsf.sort_values(by='idx', inplace=True)

        # add power surplus, where the controlled gen is removed
        sup = pd.DataFrame()
        sup['bus'] = self.bus['idx']
        sup = sup.merge(self.load[['bus', 'p0']],
                        on='bus', how='left').fillna(0).rename(columns={'p0': 'load'})
        sup['net'] = (-1 * sup.load)
        sup2 = sup[['bus', 'net']].groupby('bus').sum()
        self.line['sup'] = np.matmul(gsf_matrix, sup2.net.values)

        # --- update dict ---
        self.update_dict(model=None)

    def _default_cost(self):
        """
        Default cost data: c1=1, all other are 0.
        """
        self.cost = pd.DataFrame()
        self.cost['idx'] = self.gen['idx']
        self.cost['c2'] = 0
        self.cost['c1'] = 1
        self.cost['c0'] = 0
        self.cost['cr'] = 0

### DCOPF
Formulate DCOPF cosntraints based on base system

In [4]:
class dcopf(system):
    """
    DCOPF class.
    
    Parameters
    ----------
    name : str
        Name of the system.

    Attributes
    ----------
    bus: pandas.DataFrame
        Bus data.
    gen: pandas.DataFrame
        Generator data.
    line: pandas.DataFrame
        Line data.
    load: pandas.DataFrame
        Load data.
    gen_gsf: pandas.DataFrame
        Generator shift factor of gen bus data.
    cost: pandas.DataFrame
        Cost data.
    """

    def __init__(self, name='dcopf'):
        super().__init__(name)
        self.mdl = gb.Model(name)
        self.res_cost = 0

    def build(self):
        self.update_dict()
        # --- build DCOPF model ---
        self.mdl = self._build_vars(self.mdl)
        self.mdl = self._build_obj(self.mdl)
        self.mdl = self._build_cons(self.mdl)
        logger.warning('Successfully build DCOPF model.')

    def _build_vars(self, mdl):
        GEN = self.gendict.keys()
        # --- uncontrollable generators limit to p0 ---
        gencp = self.gen.copy()
        gencp['pmax'][gencp.ctrl == 0] = gencp['p0'][gencp.ctrl == 0]
        gencp['pmin'][gencp.ctrl == 0] = gencp['p0'][gencp.ctrl == 0]
        # --- offline geenrators limit to 0 ---
        gencp['pmax'][gencp.u == 0] = 0
        gencp['pmin'][gencp.u == 0] = 0
        # --- gen: pg ---
        self.pg = mdl.addVars(GEN, name='pg', vtype=gb.GRB.CONTINUOUS, obj=0,
                              ub=gencp.pmax.tolist(), lb=gencp.pmin.tolist())
        return mdl

    def _build_obj(self, mdl):
        GEN = self.gendict.keys()
        gendict = self.gendict
        costdict = self.costdict
        # --- minimize generation cost ---
        cost_pg = sum(self.pg[gen] * costdict[gen]['c1']
                      + self.pg[gen] * self.pg[gen] * costdict[gen]['c2']
                      + costdict[gen]['c0'] * gendict[gen]['u']  # online status
                      for gen in GEN)
        self.obj = mdl.setObjective(expr=cost_pg, sense=gb.GRB.MINIMIZE)
        return mdl

    def _build_cons(self, mdl):
        ptotal = self.load.p0.sum() 

        gendict = self.gendict
        linedict = self.linedict
        gen_gsfdict = self.gen_gsfdict

        GEN = gendict.keys()
        LINE = linedict.keys()

        # --- power balance ---
        p_sum = sum(self.pg[gen] for gen in GEN)
        mdl.addConstr(p_sum == ptotal, name='PowerBalance')

        # --- line limits ---
        for line in LINE:
            lhs1 = sum(self.pg[gen] * gen_gsfdict[gen][line] for gen in GEN)
            mdl.addConstr(lhs1+linedict[line]['sup'] <= linedict[line]['rate_a'], name=f'{line}_U')
            mdl.addConstr(lhs1+linedict[line]['sup'] >= -linedict[line]['rate_a'], name=f'{line}_D')
        return mdl

    def get_res(self):
        """
        Get resutlts, can be used after mdl.optimize().
        Returns
        -------
        DataFrame
            The output DataFrame contains setpoints ``pg``
        """
        self.build()
        self.mdl.optimize()
        # --- check if mdl is sovled ---
        if not hasattr(self.pg[self.gen.idx[0]], 'X'):
            logger.warning('DCOPF has no valid resutls!')
            pg = [0] * self.gen.shape[0]
        else:
            logger.warning('Successfully solve DCOPF.')
            # --- gather data --
            pg = []
            for gen in self.gendict.keys():
                pg.append(self.pg[gen].X)
            # --- cost ---
            self.res_cost = self.mdl.getObjective().getValue()
            logger.info(f'Total cost={np.round(self.res_cost, 3)}')
        # --- build output table ---
        dcres = pd.DataFrame()
        dcres['gen'] = self.gen['idx']
        dcres['pg'] = pg
        dcres.fillna(0, inplace=True)
        return dcres

## Test (main)


In [5]:
# get andes case from excel
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee14_base.xlsx'
case = dir_path + case_path
ssa = andes.load(case)

In [6]:
ss = dcopf()

Restricted license - for non-production use only - expires 2023-10-25


In [7]:
ss.from_andes(ssa)

In [8]:
ss.build()

Successfully build DCOPF model.


In [9]:
ss.get_res()

Successfully build DCOPF model.


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 66 rows, 12 columns and 268 nonzeros
Model fingerprint: 0xd04bc91f
Coefficient statistics:
  Matrix range     [4e-04, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 3e+00]
  RHS range        [1e-01, 4e+00]
Presolve removed 64 rows and 7 columns
Presolve time: 0.01s
Presolved: 2 rows, 5 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2370000e+00   0.000000e+00   0.000000e+00      0s
       0    2.2370000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.237000000e+00


Successfully solve DCOPF.


,gen,pg
0,PV_2,0.100
1,PV_3,0.100
2,PV_4,0.100
3,PV_5,0.100
4,PV_6,-0.100
5,Slack_1,1.937


## Test Attributes and Var

In [10]:
ss.cost

,idx,c2,c1,c0,cr
0,PV_2,0,1,0,0
1,PV_3,0,1,0,0
2,PV_4,0,1,0,0
3,PV_5,0,1,0,0
4,PV_6,0,1,0,0
5,Slack_1,0,1,0,0


In [11]:
ss.costdict

{'PV_2': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_3': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_4': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_5': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_6': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'Slack_1': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0}}

In [12]:
ss.cost['c2'] = 2

In [17]:
ss.cost

,idx,c2,c1,c0,cr
0,PV_2,2,1,0,0
1,PV_3,2,1,0,0
2,PV_4,2,1,0,0
3,PV_5,2,1,0,0
4,PV_6,2,1,0,0
5,Slack_1,2,1,0,0


In [21]:
ss.gsf_matrix.shape

(16, 14)

In [23]:
ss.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,ctrl,ramp5,ramp10,ramp30
0,PV_2,1.0,PV_2,100.0,69.0,2,0.4,0.5,0.1,1.03,1,100,200,600
1,PV_3,1.0,PV_3,100.0,69.0,3,0.4,0.5,0.1,1.01,1,100,200,600
2,PV_4,1.0,PV_4,100.0,138.0,6,0.3,1.0,0.1,1.03,1,100,200,600
3,PV_5,1.0,PV_5,100.0,69.0,8,0.3,0.5,0.1,1.03,1,100,200,600
4,PV_6,1.0,PV_6,100.0,69.0,4,-0.01,0.1,-0.1,1.01,1,100,200,600
5,Slack_1,1.0,Slack_1,100.0,69.0,1,0.81442,3.0,0.5,1.03,1,100,200,600


In [24]:
ss.pg

{'PV_2': <gurobi.Var pg[PV_2] (value 0.1)>,
 'PV_3': <gurobi.Var pg[PV_3] (value 0.1)>,
 'PV_4': <gurobi.Var pg[PV_4] (value 0.1)>,
 'PV_5': <gurobi.Var pg[PV_5] (value 0.1)>,
 'PV_6': <gurobi.Var pg[PV_6] (value -0.1)>,
 'Slack_1': <gurobi.Var pg[Slack_1] (value 1.9369999999999998)>}

In [26]:
ss.gendict.keys()

dict_keys(['PV_2', 'PV_3', 'PV_4', 'PV_5', 'PV_6', 'Slack_1'])

In [27]:
ss.costdict


{'PV_2': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_3': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_4': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_5': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'PV_6': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0},
 'Slack_1': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0}}

In [31]:
ss.load.p0.sum()

2.237

In [32]:
ss.res_cost

2.237